In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

new = cv2.imread("/home/ereverter/projects/ocr/data/raw/IMG_20230604_135247.jpg")
plt.imshow(new)

array([[[133, 134, 124],
        [104, 107,  98],
        [ 90,  92,  86],
        ...,
        [143, 133, 133],
        [148, 136, 136],
        [137, 125, 125]],

       [[132, 133, 123],
        [112, 113, 104],
        [ 93,  95,  89],
        ...,
        [142, 132, 132],
        [145, 135, 135],
        [138, 128, 128]],

       [[133, 132, 122],
        [121, 119, 111],
        [ 98,  98,  92],
        ...,
        [132, 122, 122],
        [144, 134, 134],
        [145, 135, 135]],

       ...,

       [[134, 129, 126],
        [136, 131, 128],
        [137, 132, 129],
        ...,
        [152, 147, 144],
        [156, 151, 148],
        [152, 147, 144]],

       [[143, 138, 135],
        [140, 135, 132],
        [140, 135, 132],
        ...,
        [153, 148, 145],
        [163, 158, 155],
        [164, 159, 156]],

       [[145, 140, 137],
        [139, 134, 131],
        [140, 135, 132],
        ...,
        [153, 148, 145],
        [164, 159, 156],
        [164, 159, 156]]

In [ ]:
#https://becominghuman.ai/how-to-automatically-deskew-straighten-a-text-image-using-opencv-a0c30aed83df
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    print (len(contours))
    minAreaRect = cv2.minAreaRect(largestContour)
    cv2.imwrite("temp/boxes.jpg", newImage)
    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle
# Rotate the image around its center
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage

In [ ]:
# Deskew image
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

In [ ]:
fixed = deskew(new)
cv2.imwrite("temp/rotated_fixed.jpg", fixed)

In [ ]:
display("temp/rotated_fixed.jpg")
